In [2]:
%load_ext autoreload
%autoreload 2

import dask
dask.config.set(num_workers=20)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from dask.diagnostics import Profiler, ResourceProfiler, visualize
from datetime import datetime
from satpy import Scene
from glob import glob
import numpy as np
import os

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

from satpy.utils import debug_on
#debug_on()

In [4]:
import dask
dask.config.set(scheduler='threads') 

curfile = 'D:/sat_data/ahi_main/in/0840/HS_H09_20231029_0840_B07_FLDK_R20_S0110.DAT'
tod = 'D:/sat_data/ahi_main/out/'


bbox = (-1600000, -2770000, 690000, -1040000)

comp = 'true_color_flames'

idirs = glob('D:/sat_data/ahi_main/in/*0840*')
idirs.sort()
idirs.reverse()
with dask.config.set({"array.chunk-size": "32MiB"}):
    with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
        for cdir in idirs:
            ifiles_l15 = glob(cdir+'/*.DAT')
            if len(ifiles_l15) < 5:
                continue
            curf = ifiles_l15[0]
            pos = curf.find('HS_H')
            dtstr = curf[pos+7:pos+7+13]
            outf = f'{tod}/{comp}_{dtstr}00.png'
            st = datetime.utcnow()
            #if not os.path.isfile(outf):
            if True:
                scn = Scene(ifiles_l15, reader='ahi_hsd')
                scn.load(['fire_radiative_power'], generate=False)
                #scn.load(['B07'], calibration='radiance')
                #scn.load([comp], generate=False)
                scn2 = scn#.crop(xy_bbox = bbox)
                scn3 = scn2.resample(scn2.coarsest_area(), resampler='native')
                #scn3.save_datasets(base_dir=tod,  fill_value=0, writer='simple_image')
                scn3.save_datasets(base_dir=tod, enhance=False, dtype=np.float32, fill_value=0)

            en = datetime.utcnow()
            print('\n',dtstr, (en-st).total_seconds())
    visualize([prof, rprof], show=False, save=True, filename="D:/sat_data/ahi_main/frp_new.html")

((550, 550, 550, 550, 550, 550, 550, 550, 550, 550), (825, 825, 825, 825, 825, 825, 550))
((550, 550, 550, 550, 550, 550, 550, 550, 550, 550), (825, 825, 825, 825, 825, 825, 550))
((550, 550, 550, 550, 550, 550, 550, 550, 550, 550), (825, 825, 825, 825, 825, 825, 550))


NameError: name 'klj' is not defined